In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import pickle
from pathlib import Path

DATA_FOLDER = Path("../../../data")
MODEL_FOLDER = Path("../models")
MLFLOW_DB_PATH = Path("..")

In [18]:
import mlflow

mlflow.set_tracking_uri(f"sqlite:///{MLFLOW_DB_PATH}/mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/Users/alessandro.arlandini/Documents/General DS '
 'learning/DataTalksClub/mlops-zoomcamp/ale-mlops/notebooks/03-training/experiment_tracking/mlruns/1'), creation_time=1713518478408, experiment_id='1', last_update_time=1713518478408, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [19]:
df = pd.read_parquet(DATA_FOLDER / "green_tripdata_2021-01.parquet")
df.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object

In [5]:
df["duration"] = (
    df.lpep_dropoff_datetime - df.lpep_pickup_datetime
).dt.total_seconds() / 60

In [6]:
df.duration.describe(percentiles=[0.95, 0.98, 0.99])

count    76518.000000
mean        19.927896
std         59.338594
min          0.000000
50%         13.883333
95%         44.000000
98%         56.000000
99%         67.158167
max       1439.600000
Name: duration, dtype: float64

In [7]:
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9658903787344154

In [8]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape

(73908, 21)

In [9]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]
df[categorical] = df[categorical].astype(str)

In [10]:
dv = DictVectorizer()

train_dicts = df[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

In [11]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',

DictVectorizer transforms data stored in a dictionary into a vector, in this case, a sparse matrix. It recognizes string fields and one-hot encodes them. It also recognizes numerical fields, and leaves them untouched. If we had not converted PULocation and DOLocation to strings, it would have treated them as numerical fields, which would be wrong for our application (as location numbers are only IDs, there is no notion of operation between them).

By checking the values stored in the resulting matrix one can see that except for the duration column, all the other values are 0s and 1s, consistent with the one-hot encoding. The feature names are also consistent with one-hot encoding for the `DOLocationID` and `PULocationID` fields, as every possible value originates a different column.

In [12]:
np.unique(X_train.toarray()[:, :-1].flatten())

array([0., 1.])

In [13]:
target = "duration"
y_train = df[target].values

In [14]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = lm.predict(X_train)

In [16]:
root_mean_squared_error(y_train, y_pred)

9.838799799829625

The model predictions have an RMSE of 9.5 minutes. We can probably do better, but this is still the training data. We want to compute scoring metrics on validation data too. Since we will have to repeat pretty much the same steps, we put everything together in functions for convenience.

We are building _preprocessing and training pipelines._

In [21]:
def read_dataframe(filename):
    categorical = ["PULocationID", "DOLocationID"]

    df = pd.read_parquet(filename)

    df["duration"] = (
        df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    ).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[categorical] = df[categorical].astype(str)

    return df

In [22]:
# Read training and validation dataframes
df_train = read_dataframe(DATA_FOLDER / "green_tripdata_2021-01.parquet")
df_val = read_dataframe(DATA_FOLDER / "green_tripdata_2021-02.parquet")
df_train.shape, df_val.shape

((73908, 21), (61921, 21))

In [19]:
# Engineer an interaction feature
# Interaction features combine two (or more) features into one. The idea is that the pair of values matters more than the single values together.
# In our case we will create an interaction feature from PU and DO. The interpretation is that instead of capturing what is the marginal variation due to "pick up here" and "drop off there" separately, we'll try to capture the information carried by "pick up here and drop off there".
# Note that this also reduces in half the number of features, reducing the variance of the models.
interaction = True

# In this case it suffices to combine the string to create a new categorical feature
if interaction:
    df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
    df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [20]:
# Use DictVectorizer to encode categorical features and obtain training and validation matrices
if interaction:
    # When we use the interaction features, we drop the individual features.
    categorical = ["PU_DO"]
else:
    categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [21]:
# Obtain training and validation targets
target = "duration"
y_train = df_train[target]
y_val = df_val[target]

In [22]:
# Train and validate a linear regression model
lm = LinearRegression()
lm.fit(X_train, y_train)

y_pred = lm.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.758715204520257

In [ ]:
# Train and validate a Lasso model with mlflow logging
with mlflow.start_run():
    mlflow.set_tag("developer", "alessandro")
    mlflow.log_param("train-data-path", DATA_FOLDER / "green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", DATA_FOLDER / "green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

It is common to change the code and try different configurations and parameters while experimenting. When doing that, it's important to record history to allow for proper comparisons, reproducibility and observability.

One simple method of recording what we are doing when training models is to manually log the desired quantities using mlflow. In this case we use the context environment `mlflow.start_run` to record the actions performed by the code in a single experiment run. Then we track:
* The name of the developer in a tag
* The hyperparameter `alpha`: this is a parameter that we want to tune, so it's important to keep track of the values we tried.
* The path to the training and validation data: this is a very simplified method of data versioning. We can do much better than this, but it's a start in pointing to the data used for the run.
* The resulting RMSE. This is the error metric that we obtain from the model, and it's essential to track it in order to compare runs and find the best one (that with the lowest error).

In [30]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [32]:
# we use xgb's internal DMatrix to store the data for trainig and cross-validation
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [32]:
def objective(params):
    """Take a set of parameters, train xgboost using those parameters and return
    the RMSE loss obtained with that model."""

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=15,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [33]:
# This is the search space that we want to explore with our (hyper)parameters search
# Hyperopt will loop over all the combinations of parameters that can be generated from this
# parameter space.
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 20, 1)),
    # loguniform generates values whose logarithms are uniformly spaced.
    # In this case the logarithms will be -3, -2, -1, 0
    # so the values will be exp(-3), exp(-2), exp(-1), 1
    "learning_rate": hp.loguniform("learning_rate", -3, 0),
    "reg_alpha": hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# fmin minimizes a function over a domain, or more precisely finds the minimum value of a function
# over a search space (trying only the inputs specified in the search space).
# By viewing our model as a function (params) -> (loss) we can use fmin to find the set of hyperparameters
# that minimizes the loss, thus finding the best model.
best_result = fmin(
    # we wrote the "objective" function to take a set of hyperparams and returning the loss
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=40,
    # storage for iterations and results
    trials=Trials()
)

[0]	validation-rmse:11.75673                          
[1]	validation-rmse:11.33466                          
[2]	validation-rmse:10.94414                          
  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:06:44] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:10.58347                          
[4]	validation-rmse:10.25118                          
[5]	validation-rmse:9.94511                           
[6]	validation-rmse:9.66379                           
[7]	validation-rmse:9.40561                           
[8]	validation-rmse:9.16844                           
[9]	validation-rmse:8.95131                           
[10]	validation-rmse:8.75266                          
[11]	validation-rmse:8.57070                          
[12]	validation-rmse:8.40437                          
[13]	validation-rmse:8.25221                          
[14]	validation-rmse:8.11396                          
[15]	validation-rmse:7.98707                          
[16]	validation-rmse:7.87161                          
[17]	validation-rmse:7.76634                          
[18]	validation-rmse:7.67016                          
[19]	validation-rmse:7.58359                          
[20]	validation-rmse:7.50406                          
[21]	valid

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:06:47] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:9.09541                                                    
[3]	validation-rmse:8.49709                                                    
[4]	validation-rmse:8.04644                                                    
[5]	validation-rmse:7.71006                                                    
[6]	validation-rmse:7.45886                                                    
[7]	validation-rmse:7.27343                                                    
[8]	validation-rmse:7.13699                                                    
[9]	validation-rmse:7.03357                                                    
[10]	validation-rmse:6.95608                                                   
[11]	validation-rmse:6.89827                                                   
[12]	validation-rmse:6.85468                                                   
[13]	validation-rmse:6.82133                                                   
[14]	validation-rmse:6.79526            

...

[1]	validation-rmse:10.63236                                                    
[2]	validation-rmse:10.00499                                                    
[3]	validation-rmse:9.46983                                                     
[4]	validation-rmse:9.01274                                                     
[5]	validation-rmse:8.62726                                                     
[6]	validation-rmse:8.30292                                                     
[7]	validation-rmse:8.03034                                                     
[8]	validation-rmse:7.80268                                                     
[9]	validation-rmse:7.61021                                                     
[10]	validation-rmse:7.45183                                                    
[11]	validation-rmse:7.31781                                                    
[12]	validation-rmse:7.20633                                                    
[13]	validation-rmse:7.11434

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:08:37] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:10.76076                                                    
[4]	validation-rmse:10.45624                                                    
[5]	validation-rmse:10.17231                                                    
[6]	validation-rmse:9.90808                                                     
[7]	validation-rmse:9.66230                                                     
[8]	validation-rmse:9.43364                                                     
[9]	validation-rmse:9.22154                                                     
[10]	validation-rmse:9.02530                                                    
[11]	validation-rmse:8.84326                                                    
[12]	validation-rmse:8.67464                                                    
[13]	validation-rmse:8.51902                                                    
[14]	validation-rmse:8.37511                                                    
[15]	validation-rmse:8.24226

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:08:41] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:6.91707                                                     
[4]	validation-rmse:6.78676                                                     
[5]	validation-rmse:6.72106                                                     
[6]	validation-rmse:6.68491                                                     
[7]	validation-rmse:6.66770                                                     
[8]	validation-rmse:6.65334                                                     
[9]	validation-rmse:6.64257                                                     
[10]	validation-rmse:6.63878                                                    
[11]	validation-rmse:6.63440                                                    
[12]	validation-rmse:6.63002                                                    
[13]	validation-rmse:6.62577                                                    
[14]	validation-rmse:6.62303                                                    
[15]	validation-rmse:6.62118

In [35]:
# Training of the best model as inferred from the mlflow ui, by choosing the model that minimizes the RMSE loss
params = {
    "learning_rate": 0.6424019772458974,
    "max_depth": 20,
    "min_child_weight": 2.2694144028711833,
    "objective": "reg:linear",
    "reg_alpha": 0.025551415216516424,
    "reg_lambda": 0.009147735459264332,
    "seed": 42,
}

# Log using autologging, which automatically logs plenty info about the run
mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, "validation")],
    early_stopping_rounds=15,
)

2024/04/19 16:27:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b718c906f65f418fb57e0bc5c2b58959', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:7.79040


/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:27:53] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:6.87496
[2]	validation-rmse:6.67857
[3]	validation-rmse:6.62456
[4]	validation-rmse:6.60289
[5]	validation-rmse:6.59332
[6]	validation-rmse:6.57983
[7]	validation-rmse:6.57435
[8]	validation-rmse:6.56953
[9]	validation-rmse:6.56221
[10]	validation-rmse:6.55691
[11]	validation-rmse:6.55156
[12]	validation-rmse:6.54857
[13]	validation-rmse:6.54374
[14]	validation-rmse:6.53648
[15]	validation-rmse:6.53111
[16]	validation-rmse:6.52934
[17]	validation-rmse:6.52656
[18]	validation-rmse:6.52193
[19]	validation-rmse:6.51718
[20]	validation-rmse:6.51076
[21]	validation-rmse:6.50723
[22]	validation-rmse:6.50659
[23]	validation-rmse:6.50324
[24]	validation-rmse:6.49680
[25]	validation-rmse:6.49462
[26]	validation-rmse:6.49068
[27]	validation-rmse:6.48778
[28]	validation-rmse:6.48296
[29]	validation-rmse:6.48014
[30]	validation-rmse:6.47863
[31]	validation-rmse:6.47687
[32]	validation-rmse:6.47484
[33]	validation-rmse:6.47093
[34]	validation-rmse:6.46951
[35]	validation-rmse:6.

2024/04/19 16:28:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/04/19 16:28:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:28:01] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


### Model management

This tracks what is happening during model training, but it does not track the model itself. This can also be done in mlflow, which has the following benefits compared to folder-based logging:
* automated instead of manual (less error-prone)
* reliable versioning
* model lineage (users can retrieve how the model was trained: parameters, dataset, etc.)

In [ ]:
# First approach at logging the model itself in mlflow: logging as an artifact
mlflow.log_artifact(MODEL_FOLDER / 'lin_reg.bin', artifact_path='models_pickle')

In [35]:
# Second approach: log the model inside of a run, using the appropriate mlflow API

with mlflow.start_run():
    # Training of the best model as inferred from the mlflow ui, by choosing the model that minimizes the RMSE loss
    params = {
        "learning_rate": 0.6424019772458974,
        "max_depth": 20,
        "min_child_weight": 2.2694144028711833,
        "objective": "reg:linear",
        "reg_alpha": 0.025551415216516424,
        "reg_lambda": 0.009147735459264332,
        "seed": 42,
    }
    
    mlflow.log_params(params)


    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=15,
    )
    
    # Logging it as a model using the mlflow API for the model framework (xgboost)
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')
    
    # Logging the preprocessor as an artifact
    # First we save it as pickle, and then we log it by referencing the path
    with open(MODEL_FOLDER / 'preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact(MODEL_FOLDER / "preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:7.79040


/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:11:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:6.87496
[2]	validation-rmse:6.67857
[3]	validation-rmse:6.62456
[4]	validation-rmse:6.60289
[5]	validation-rmse:6.59332
[6]	validation-rmse:6.57983
[7]	validation-rmse:6.57435
[8]	validation-rmse:6.56953
[9]	validation-rmse:6.56221
[10]	validation-rmse:6.55691
[11]	validation-rmse:6.55156
[12]	validation-rmse:6.54857
[13]	validation-rmse:6.54374
[14]	validation-rmse:6.53648
[15]	validation-rmse:6.53111
[16]	validation-rmse:6.52934
[17]	validation-rmse:6.52656
[18]	validation-rmse:6.52193
[19]	validation-rmse:6.51718
[20]	validation-rmse:6.51076
[21]	validation-rmse:6.50723
[22]	validation-rmse:6.50659
[23]	validation-rmse:6.50324
[24]	validation-rmse:6.49680
[25]	validation-rmse:6.49462
[26]	validation-rmse:6.49068
[27]	validation-rmse:6.48778
[28]	validation-rmse:6.48296
[29]	validation-rmse:6.48014
[30]	validation-rmse:6.47863
[31]	validation-rmse:6.47687
[32]	validation-rmse:6.47484
[33]	validation-rmse:6.47093
[34]	validation-rmse:6.46951
[35]	validation-rmse:6.

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:11:59] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/al

In the above code we also logged the `DictVectorizer`: this is a preprocessor, not part of the model. It is used to generate new features from the raw data, that are then used to train the model.

In the future, when we want to make predictions with the model and we are given raw data as input, we want to preprocess that data in the same way that we did with the training data. It is then essential that we log the preprocessor for future use. We log it as an artifact.

In order to make predictions, we load the model from mlflow. In our case there are two different "flavours" that we can use: `pyfunc` which means to load it as a generic python function, and `xgboost` which loads the model as an xgboost object (giving back the original object).

In [36]:
# Load model as a PyFuncModel.
logged_model = 'runs:/0ebd7485b47c43ab9b53ab72548bccde/models_mlflow'

loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:17:30] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 0ebd7485b47c43ab9b53ab72548bccde

In [38]:
# Load model as a xgboost model
xgboost_model = mlflow.xgboost.load_model(logged_model)

xgboost_model

/Users/alessandro.arlandini/Documents/General DS learning/DataTalksClub/mlops-zoomcamp/venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:18:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [40]:
y_pred = xgboost_model.predict(valid)
y_pred[:10]

array([15.175811 ,  7.352664 , 17.564875 , 24.57404  ,  9.3976755,
       17.19716  , 10.984803 ,  8.377045 ,  8.945719 , 19.10443  ],
      dtype=float32)